## Define sub functions that will be called on each query / match

In [2]:
#find all indexs for Query; the find function only find first occurence
#s: the long string
#c: the cetain pattern: in our case, is either "Query=" or "> ID"
#return a list of indexs
#NOTE: no need to call this directly; already called in sepString(s,c)
def findAllSubstring(s,c):
    pos = [] #return this list that stores all index
    idx = s.find(c)
    while idx != -1: #when there's more to find
        pos.append(idx)
        idx = s.find(c, idx + 1)
    return pos

#cut a long string into several parts base on certain pattern
#s: the long string
#c: the cetain pattern: in our case, is either "Query=" or "> ID"
#return a list of substrings
def sepString(s,c):
    pos = findAllSubstring(s,c) #find the positions; seperate base on them
    subs = []
    i = 0
    while (i < (len(pos)-1)):
        start,end = pos[i], pos[i+1]
        subs.append(s[start:end])
        i += 1
    try:
        subs.append(s[pos[len(pos)-1]:len(s)]) #append the last element
    except:
        print(len(pos))    
    return subs

## Import data

In [3]:
#prepare 1: a list that records the small proteins' IDs
#NOTE: "small" proteins are proteins with less than 100 AA. 
small_proteins = ['cds-YP_009724392.1','cds-YP_009724394.1','cds-YP_009725318.1','cds-YP_009725255.1']

In [4]:
#read the file as big string; I manually deleted the header
from pathlib import Path
file=Path('blastx_e-2_modified.txt').read_text()
#seperate the string by “Query=” so we can investigate the best alignment for each sample separately
queries = sepString(file,'Query=')

## Do mutation hunt

In [16]:
Mutations = [] #NOTE: use this to store mutations

test = queries[0:1] #a small dataset that only contains 1 query. TODO: delete this var if you don't want to use it

#for each query, check all IDs (ie, check all proteins);
for query in queries: 
    #-----get a list of all IDs-------#
    IDs = sepString(query,'> ID=')
    #the last element in IDs needs modification: it contains the alpha, k, lambda val, etc
    x = IDs[len(IDs) - 1].find('Lambda')
    newLastEle = IDs[len(IDs) - 1][0:x]
    IDs = IDs[0:len(IDs)-1] #remove the last element
    IDs.append(newLastEle)

    #--------At this point, we get all IDs-----------#
    for ID in IDs:
        content = ID.split('\n') ##NOTE: this is the variable that stores all line for one match, including the header/scores/actual alignment
        #remove all empty string in list
        content[:] = [v for v in content if v != ""]

        #find which element starts with 'length='
        before = 0
        for i in range(len(content)):
            if('Length=' in content[i]):
                before = i
                break

        #get the protein header
        header = ''
        for i in range(before):
            header += content[i]
        protein_id = header.split(';')[0] #this is the protein ID that starts with YP
        protein_id = protein_id.replace('> ID=', '') # formatting to match the item in small_proteins

        #get the identities and positives 
        score = content[before+1] + content[before+2] + content[before+3]
        score = score.split(',')
        Identities = score[2].split('.')[1]
        Positives = score[3]
        #print(Identities,Positives) 

        # get the number of bases that match and the total number of bases
        score_i = Identities.split(' ')[3]
        score_p = Positives.split(' ')[3]
        # this is for score_i, what about score_p?
        num_base_match = int(score_i.split('/')[0])
        num_base_total = int(score_i.split('/')[1])

        # check if enough mutation
        num_mutations = num_base_total - num_base_match
        counter = 0
        
        # ignore the small protein with more than 2 mutations
        if protein_id in small_proteins:
            if num_mutations > 2:
                continue

        # skip this ID if the two numbers above match (no mutation)
        if num_base_match == num_base_total:
            continue

        else:
            # start from the alignemnts
            start_seq_pos = before + 4

            for i in range(start_seq_pos, len(content), 3):

                # get query_seq and subjct_seq
                query_seq_line = content[i]
                # print(query_seq_line)         # checking
                sbjct_seq_line = content[i+2]
                query_seqs = query_seq_line.split(' ')
                query_seqs = [item for item in query_seqs if item != '']
                query_seq = query_seqs[2]
                sbjct_seqs = sbjct_seq_line.split(' ')
                sbjct_seqs = [item for item in sbjct_seqs if item != '']
                sbjct_seq = sbjct_seqs[2]
                
                # change to upper case
                query_seq = query_seq.upper()
                sbjct_seq = sbjct_seq.upper()

                # no mutation
                if query_seq == sbjct_seq:
                    continue

                else:
                    for i in range(len(query_seq)):
                        # if same char, no mutation
                        if query_seq[i] == sbjct_seq[i]:
                            continue
                        # if X-R, no mutation
                        elif query_seq[i] == 'X' and sbjct_seq[i].isalpha():
                            counter += 1 # X-R also counts as a difference in "identities"
                            continue
                        # else, mutation occurs
                        else:
                            counter += 1
                            mut_pt = int(sbjct_seqs[1]) + i # mutation point
                            # print(mut_pt)                 # checking
                            db_AA = sbjct_seq[i] # AA acid that differs in the db and the query
                            query_AA = query_seq[i] 

                            # formatting
                            Mutations.append(f'{protein_id}:{db_AA}{mut_pt}{query_AA}')

                            # find enough mutation, break inner loop - char by char
                            if counter < num_mutations:
                                continue
                            else:
                                break

                # find enough mutation, break outer loop - three lines a time            
                if counter < num_mutations:
                    continue
                else:
                    break
    # The 2 breaks deleted to do this for all queries 

## Summarize results and output to file

In [45]:
# sort the mutation
import collections

#count the frequency of each mutation 
occurrences = collections.Counter(Mutations)
d = {}
for key, value in occurrences.items():
    d[key] = value
#sort mutation by most frequent to least frequent
freq = sorted([(value,key) for (key,value) in d.items()],reverse=True)
#write result to file
f = open("MutationHuntRes.txt", "a")
for item in freq:
    s = item[1] + ":" + str(item[0]) + '\n'
    f.write(s) 
f.close()
